In [24]:
import pandas as pd
from sklearn.model_selection import train_test_split
import numpy as np
from sklearn.metrics import accuracy_score
from keras.layers import LSTM, Dense,Input,Layer
from keras import backend as K,Model
from numpy.random import seed
from imblearn.under_sampling import RandomUnderSampler 
seed(0)
import tensorflow as tf
tf.random.set_seed(0)

In [25]:
data = pd.read_csv('creditcard.csv', na_filter=True)

data.drop(['Time'], axis=1, inplace=True)
selected_vars =['V4','V14','V10','V16','V9','V12','V11','V17','V7','V21','V18','V8','V1','V3','V6','V2','V27','V5','V26','Class']


In [26]:
tr_data = data[selected_vars]
tr_data.shape

(284807, 20)

In [27]:
X = tr_data.drop(['Class'], axis = 'columns')
Label_Data = tr_data['Class']

In [28]:
y=tr_data['Class']

In [29]:
from sklearn.preprocessing import StandardScaler
## Standardizing the data
X=pd.DataFrame(StandardScaler().fit_transform(X))

In [30]:
X_train0,X_test,y_train0,y_test = train_test_split(X, y, test_size=0.3, shuffle=False,random_state=0)
X_train0,X_train1,y_train0,y_train1 = train_test_split(X_train0, y_train0, test_size=0.5, shuffle=False,random_state=0)

from sklearn.linear_model import SGDOneClassSVM
df1_normal=X_train0[y_train0==0]
clf = SGDOneClassSVM(nu=0.02,random_state=0)
res = clf.fit_predict(df1_normal)
res[res==1] = 0
res[res==-1] = 1
ros = RandomUnderSampler(sampling_strategy=0.95)
ros.fit_resample(df1_normal,res)
index=ros.sample_indices_
X_train0.drop(X_train0.index[index], inplace=True)
y_train0.drop(y_train0.index[index], inplace=True)

df1_normal=X_train1[y_train1==1]
clf = SGDOneClassSVM(nu=0.035,random_state=0)
res = clf.fit_predict(df1_normal)
res[res==1] = 0
res[res==-1] = 1
ros = RandomUnderSampler(sampling_strategy=0.95)
ros.fit_resample(df1_normal,res)
index=ros.sample_indices_
X_train1.drop(X_train1.index[index], inplace=True)
y_train1.drop(y_train1.index[index], inplace=True)

In [31]:
X_train0.shape,X_train1.shape

((95992, 19), (99676, 19))

In [32]:
y_train0.shape,y_train1.shape

((95992,), (99676,))

In [33]:
X_test.shape

(85443, 19)

# N=1

In [34]:
# X_train et X_test sont des dataframe qui contient les features
train_LSTM_X0=X_train0.to_numpy()
test_LSTM_X=X_test.to_numpy()
train_LSTM_y0=y_train0.to_numpy()
test_LSTM_y=y_test.to_numpy()
w=3
X_train0=np.zeros([train_LSTM_X0.shape[0]-w+1,w, train_LSTM_X0.shape[1]])
for i in range(train_LSTM_X0.shape[0]-w+1):
    for j in range(w):
        X_train0[i][j]=train_LSTM_X0[i+j]
X_test=np.zeros([test_LSTM_X.shape[0]-w+1,w, test_LSTM_X.shape[1]])
for i in range(test_LSTM_X.shape[0]-w+1):
    for j in range(w):
        X_test[i][j]=test_LSTM_X[i+j]
## Recuperation des labels

y_train0=np.delete(train_LSTM_y0,[*range(0, w-1, 1)], 0)
y_test=np.delete(test_LSTM_y,[*range(0, w-1, 1)], 0)

train_LSTM_X0=X_train0
test_LSTM_X=X_test
train_LSTM_y0=y_train0
test_LSTM_y=y_test

In [35]:
inputs=Input((w,19))
x1=LSTM(50)(inputs)
outputs=Dense(1,activation='sigmoid')(x1)
model0=Model(inputs,outputs)

In [36]:
model0.compile(loss='binary_crossentropy', optimizer='adam', metrics=['Precision','Recall'])

In [37]:
history0=model0.fit(train_LSTM_X0, train_LSTM_y0,epochs=4,batch_size=400,validation_data=(test_LSTM_X, test_LSTM_y))

Epoch 1/4
240/240 [==============================] - 7s 18ms/step - loss: 0.1246 - precision: 0.0098 - recall: 0.1065 - val_loss: 0.1264 - val_precision: 0.0180 - val_recall: 0.2870
Epoch 2/4
240/240 [==============================] - 3s 12ms/step - loss: 0.0057 - precision: 0.7616 - recall: 0.5324 - val_loss: 0.0527 - val_precision: 0.3860 - val_recall: 0.6111
Epoch 3/4
240/240 [==============================] - 2s 10ms/step - loss: 0.0044 - precision: 0.8122 - recall: 0.7407 - val_loss: 0.0292 - val_precision: 0.8211 - val_recall: 0.7222
Epoch 4/4
240/240 [==============================] - 3s 11ms/step - loss: 0.0039 - precision: 0.8255 - recall: 0.8102 - val_loss: 0.0208 - val_precision: 0.9302 - val_recall: 0.7407


In [38]:
# predict probabilities for test set
yhat_probs0 = model0.predict(test_LSTM_X, verbose=0)
# reduce to 1d array
yhat_probs0 = yhat_probs0[:, 0]> 0.5

In [39]:
# demonstration of calculating metrics for a neural network model using sklearn
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
# accuracy: (tp + tn) / (p + n)
accuracy = accuracy_score(test_LSTM_y, yhat_probs0)
print('Accuracy: %f' % accuracy)
# precision tp / (tp + fp)
precision = precision_score(test_LSTM_y, yhat_probs0)
print('Precision: %f' % precision)
# recall: tp / (tp + fn)
recall = recall_score(test_LSTM_y, yhat_probs0)
print('Recall: %f' % recall)
f1 = f1_score(test_LSTM_y, yhat_probs0)
print('f1: %f' % f1)

Accuracy: 0.999602
Precision: 0.930233
Recall: 0.740741
f1: 0.824742


# N=2

In [40]:
# X_train et X_test sont des dataframe qui contient les features
train_LSTM_X1=X_train1.to_numpy()
train_LSTM_y1=y_train1.to_numpy()
w=3
X_train1=np.zeros([train_LSTM_X1.shape[0]-w+1,w, train_LSTM_X1.shape[1]])
for i in range(train_LSTM_X1.shape[0]-w+1):
    for j in range(w):
        X_train1[i][j]=train_LSTM_X1[i+j]

y_train1=np.delete(train_LSTM_y1,[*range(0, w-1, 1)], 0)
train_LSTM_X1=X_train1
train_LSTM_y1=y_train1

In [41]:
inputs=Input((w,19))
x1=LSTM(50)(inputs)
outputs=Dense(1,activation='sigmoid')(x1)
model1=Model(inputs,outputs)

In [42]:
model1.compile(loss='binary_crossentropy', optimizer='adam', metrics=['Precision','Recall'])

In [43]:
history1=model1.fit(train_LSTM_X1, train_LSTM_y1,epochs=20,batch_size=400,validation_data=(test_LSTM_X, test_LSTM_y))

Epoch 1/20
250/250 [==============================] - 6s 13ms/step - loss: 0.1281 - precision: 0.0165 - recall: 0.1988 - val_loss: 0.0058 - val_precision: 1.0000 - val_recall: 0.0093
Epoch 2/20
250/250 [==============================] - 2s 10ms/step - loss: 0.0048 - precision: 0.9535 - recall: 0.5093 - val_loss: 0.0039 - val_precision: 1.0000 - val_recall: 0.4259
Epoch 3/20
250/250 [==============================] - 3s 11ms/step - loss: 0.0039 - precision: 0.9266 - recall: 0.6273 - val_loss: 0.0033 - val_precision: 1.0000 - val_recall: 0.5370
Epoch 4/20
250/250 [==============================] - 3s 11ms/step - loss: 0.0035 - precision: 0.9048 - recall: 0.7081 - val_loss: 0.0031 - val_precision: 0.9857 - val_recall: 0.6389
Epoch 5/20
250/250 [==============================] - 3s 11ms/step - loss: 0.0033 - precision: 0.9062 - recall: 0.7205 - val_loss: 0.0029 - val_precision: 0.9859 - val_recall: 0.6481
Epoch 6/20
250/250 [==============================] - 3s 10ms/step - loss: 0.0031 - p

In [44]:
# predict probabilities for test set
yhat_probs1 = model1.predict(test_LSTM_X, verbose=0)
# reduce to 1d array
yhat_probs1 = yhat_probs1[:, 0]> 0.5

In [45]:
# accuracy: (tp + tn) / (p + n)
accuracy = accuracy_score(test_LSTM_y, yhat_probs1)
print('Accuracy: %f' % accuracy)
# precision tp / (tp + fp)
precision = precision_score(test_LSTM_y, yhat_probs1)
print('Precision: %f' % precision)
# recall: tp / (tp + fn)
recall = recall_score(test_LSTM_y, yhat_probs1)
print('Recall: %f' % recall)
f1 = f1_score(test_LSTM_y, yhat_probs1)
print('f1: %f' % f1)

Accuracy: 0.999661
Precision: 1.000000
Recall: 0.731481
f1: 0.844920


# FFNN

In [46]:
y_probs0 = model0.predict(train_LSTM_X0, verbose=0)
y_probs1 = model1.predict(train_LSTM_X0, verbose=0)
y_probs00 = model0.predict(train_LSTM_X1, verbose=0)
y_probs11 = model1.predict(train_LSTM_X1, verbose=0)

In [47]:
X_train0=np.concatenate((y_probs0,y_probs1),axis=1)
X_train1=np.concatenate((y_probs00,y_probs11),axis=1)
X_train=np.concatenate((X_train0,X_train1),axis=0)
y_train=np.concatenate((y_train0,y_train1),axis=0)
X_train.shape,y_train.shape

((195664, 2), (195664,))

In [48]:
from sklearn.neural_network import MLPClassifier
clf = MLPClassifier().fit(X_train, y_train)

# evaluation

In [49]:
yhat_probs0 = model0.predict(test_LSTM_X, verbose=0)
yhat_probs1 = model1.predict(test_LSTM_X,verbose=0)
X_test=np.concatenate((yhat_probs0,yhat_probs1),axis=1)
y_pred=clf.predict(X_test)

In [50]:
# accuracy: (tp + tn) / (p + n)
accuracy = accuracy_score(test_LSTM_y, y_pred)
print('Accuracy: %f' % accuracy)
# precision tp / (tp + fp)
precision = precision_score(test_LSTM_y, y_pred)
print('Precision: %f' % precision)
# recall: tp / (tp + fn)
recall = recall_score(test_LSTM_y, y_pred)
print('Recall: %f' % recall)
#F1 = 2 * (precision * recall) / (precision + recall)
f1=f1_score(test_LSTM_y, y_pred)
print('f1: %f' % f1)

Accuracy: 0.999661
Precision: 1.000000
Recall: 0.731481
f1: 0.844920
